In [1]:
!pip install -q transformers datasets sentencepiece coral_pytorch

     |████████████████████████████████| 3.8 MB 4.1 MB/s 
     |████████████████████████████████| 325 kB 72.7 MB/s 
     |████████████████████████████████| 1.2 MB 20.1 MB/s 
     |████████████████████████████████| 6.5 MB 51.1 MB/s 
     |████████████████████████████████| 596 kB 63.4 MB/s 
     |████████████████████████████████| 895 kB 55.5 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 134 kB 67.8 MB/s 
     |████████████████████████████████| 212 kB 70.7 MB/s 
     |████████████████████████████████| 1.1 MB 57.8 MB/s 
     |████████████████████████████████| 127 kB 74.3 MB/s 
     |████████████████████████████████| 271 kB 64.0 MB/s 
     |████████████████████████████████| 94 kB 4.7 MB/s 
     |████████████████████████████████| 144 kB 74.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6

In [3]:
import torch
import torch.nn as nn
from torch.functional import F

from datasets import Dataset

import transformers as ts
from transformers import AutoTokenizer , AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from transformers import create_optimizer

from transformers.file_utils import ModelOutput
from transformers.modeling_outputs import SequenceClassifierOutput

from coral_pytorch.layers import CoralLayer
from coral_pytorch.losses import coral_loss
from coral_pytorch.dataset import levels_from_labelbatch
from coral_pytorch.dataset import proba_to_label

from dataclasses import dataclass
from typing import Optional, Tuple

import numpy as np
import pandas as pd
from scipy import stats

import sys

In [4]:
from data_loader import (
    retrieve_instances_from_dataset,
    retrieve_labels_from_dataset_for_classification,
    retrieve_labels_from_dataset_for_ranking,
    write_predictions_to_file,
)

In [5]:
PRE_TRAINED_MODEL = "microsoft/deberta-v3-base"
DIM_KEY = "hidden_size"

#Preparing Data

In [6]:
def loadDataset(dataPath , labelPath=None , scoresPath=None):
  dataset = pd.read_csv(dataPath, sep="\t", quoting=3)
  
  ids , sentences , fillers = retrieve_instances_from_dataset(dataset)

  #Creating dictionaries to convert datas to Huggingface Dataset
  datasetDict = {
      "id": ids,
      "sentence": sentences,
      "filler": fillers,
  }

  labels = None
  
  if labelPath != None:
    labels = pd.read_csv(labelPath, sep="\t", header=None, names=["Id", "Label"])
    labels = retrieve_labels_from_dataset_for_classification(labels)
    datasetDict["labels"] = labels

  scores = None

  if scoresPath != None:
    scores = pd.read_csv(scoresPath, sep="\t", header=None, names=["Id", "Label"])
    scores = retrieve_labels_from_dataset_for_ranking(scores)
    datasetDict["scores"] = scores

  #Removing Periods if fillers appear at the end of the sentence (because if we don't period will be considered last word piece of the filler)
  for index , _ in enumerate(fillers):
    fillers[index].replace("." , "")

  #Creating Huggingface Datasets from Dictionaries
  dataset = Dataset.from_dict(datasetDict)

  return dataset

In [7]:
trainDataset = loadDataset("Data/Train_Dataset.tsv",
                           "Data/Train_Labels.tsv", 
                           "Data/Train_Scores.tsv")

valDataset = loadDataset("Data/Val_Dataset",
                         "Data/Val_Labels", 
                         "Data/Val_Scores")

testDataset = loadDataset("Data/Test_Scores")

In [8]:
print(trainDataset)
print(valDataset)
print(testDataset)

Dataset({
    features: ['id', 'sentence', 'filler', 'labels', 'scores'],
    num_rows: 19975
})
Dataset({
    features: ['id', 'sentence', 'filler', 'labels', 'scores'],
    num_rows: 2500
})
Dataset({
    features: ['id', 'sentence', 'filler'],
    num_rows: 2500
})


In [9]:
print(trainDataset["sentence"][0])

Resolved pattern: IMPLICIT REFERENCE
Section header: Things You'll Need
Article title: How to Talk to Your Partner About Their Weigh
Text: (...)   * Lose Weight the Healthy Way   * Use Exercise to Help You Lose Weight * Lose Weight Fast with [MASK] 


#Preprocessing

In [10]:
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer , return_tensors="pt")

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/579 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
def preprocessDataset(dataset , tokenizer):
  def addToDict(dict_1 , dict_2 , columns_1=[] , columns_2=["input_ids" , "attention_mask"]):
    for item_1 , item_2 in zip(columns_1 , columns_2):
      dict_1[item_1] = dict_2.pop(item_2)

  def mappingFunction(dataset):
    outputDict = {}

    cleanedSentence = dataset["sentence"].replace("\n" , " ").replace("(...)" , "").strip()
    sentenceWithFiller = cleanedSentence.replace("[MASK]" , dataset["filler"].strip()).strip()

    tokenized_sentence = tokenizer(sentenceWithFiller)
    addToDict(outputDict , tokenized_sentence , ["input_ids" , "attention_mask"])

    #Getting the index of the last word piece of the filler
    if "cls_token" in tokenizer.special_tokens_map.keys():
      filler_indecies = len(tokenizer(tokenizer.special_tokens_map["cls_token"] + " " + cleanedSentence.split("[MASK]")[0].strip() + " " + dataset["filler"].strip() , add_special_tokens=False)["input_ids"]) - 1
    elif "bos_token" in tokenizer.special_tokens_map.keys():
      filler_indecies = len(tokenizer(tokenizer.special_tokens_map["bos_token"] + " " + cleanedSentence.split("[MASK]")[0].strip() + " " + dataset["filler"].strip() , add_special_tokens=False)["input_ids"]) - 1
    else:
      filler_indecies = len(tokenizer(cleanedSentence.split("[MASK]")[0].strip() + " " + dataset["filler"].strip() , add_special_tokens=False)["input_ids"]) - 1

    outputDict["filler_indecies"] = filler_indecies

    return outputDict
  
  return dataset.map(mappingFunction , batched=False)

In [12]:
tokenizedTrainDataset = preprocessDataset(trainDataset , tokenizer)
tokenizedValDataset = preprocessDataset(valDataset , tokenizer)
tokenizedTestDataset = preprocessDataset(testDataset , tokenizer)

  0%|          | 0/19975 [00:00<?, ?ex/s]

  0%|          | 0/2500 [00:00<?, ?ex/s]

  0%|          | 0/2500 [00:00<?, ?ex/s]

In [13]:
print(tokenizedTrainDataset)
print(tokenizedValDataset)
print(tokenizedTestDataset)

Dataset({
    features: ['id', 'sentence', 'filler', 'labels', 'scores', 'input_ids', 'attention_mask', 'filler_indecies'],
    num_rows: 19975
})
Dataset({
    features: ['id', 'sentence', 'filler', 'labels', 'scores', 'input_ids', 'attention_mask', 'filler_indecies'],
    num_rows: 2500
})
Dataset({
    features: ['id', 'sentence', 'filler', 'input_ids', 'attention_mask', 'filler_indecies'],
    num_rows: 2500
})


#Model Definition

In [14]:
@dataclass
class CustomOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    classificationOutput: torch.FloatTensor = None
    regressionOutput: torch.FloatTensor = None

In [15]:
class SequenceClassificationModel(nn.Module):
  def __init__(self,
               encoder,
               dim, 
               use_coral=False, 
               use_cls=True, 
               supportPooledRepresentation=False, 
               mode="both", 
               num_labels=3, 
               num_ranks=5,
               lambda_c=0.5,
               lambda_r=0.5,
               dropout_rate=0.2):
    
    super().__init__()

    #mode can be one of these: ["both" , "classification" , "regression"]

    self.encoder = encoder
    self.dim = dim
    self.use_coral = use_coral
    self.use_cls = use_cls
    self.supportPooledRepresentation = supportPooledRepresentation
    self.mode = mode
    self.num_labels = num_labels
    self.num_ranks = num_ranks
    self.lambda_c = lambda_c
    self.lambda_r = lambda_r
    self.dropout_rate = dropout_rate

    if self.use_cls:
      self.pre_classifier = nn.Linear(self.dim*2 , self.dim , bias=True)
    else:
      self.pre_classifier = nn.Linear(self.dim , self.dim , bias=True)

    self.dropout = nn.Dropout(p=self.dropout_rate , inplace=False)

    self.regressionHead = CoralLayer(self.dim , self.num_ranks)

    if use_coral:
      self.classificationHead = CoralLayer(self.dim , self.num_labels)
    else:
      self.classificationHead = nn.Linear(self.dim , self.num_labels , bias=True)
    
  def forward(
      self,
      input_ids,
      attention_mask,
      filler_indecies,
      labels=None,
      scores=None,
      **args):
    
    device = self.encoder.device

    # Getting fillers representation from pre-trained transformer (encoder)
    sentence_embedding = self.encoder(
        input_ids=input_ids,
        attention_mask=attention_mask,
    )

    #Getting Fillers Representation
    filler_tokens = sentence_embedding[0][filler_indecies[0] , filler_indecies[1]]
    fillers = filler_tokens[: , 0 , :]

    #Concatenating [CLS] output with Filler output if the model supports [CLS]
    pooled_output = None

    if self.use_cls:
      if self.supportPooledRepresentation:
        pooled_output = torch.concat((sentence_embedding[1] , fillers) , dim=-1)
      else:
        pooled_output = torch.concat((sentence_embedding[0][: , 0 , :] , fillers) , dim=-1)
    else:
      pooled_output = fillers
    
    #Passing Pooled Output to another dense layer followed by activation function and dropout
    pooled_output = self.pre_classifier(pooled_output)  
    pooled_output = nn.GELU()(pooled_output)  
    pooled_output = self.dropout(pooled_output)
    
    #Passing the final output to the classificationHead and RegressionHead
    classificationOutput = self.classificationHead(pooled_output)
    regressionOutput = self.regressionHead(pooled_output)

    totalLoss = None
    classification_loss = None
    regression_loss = None

    #Computing classification loss
    if labels != None and (self.mode.lower() == "both" or self.mode.lower() == "classification"):
      if self.use_coral:
        levels = levels_from_labelbatch(labels.view(-1) , self.num_labels).to(device)
        classification_loss = coral_loss(classificationOutput.view(-1 , self.num_labels - 1) , levels.view(-1 , self.num_labels - 1))
      else:
        loss_fct = nn.CrossEntropyLoss()
        classification_loss = loss_fct(classificationOutput.view(-1 , self.num_labels) , labels.view(-1))

    #Computing regression loss
    if scores != None and (self.mode.lower() == "both" or self.mode.lower() == "regression"):
      levels = levels_from_labelbatch(scores.view(-1) , self.num_ranks).to(device)
      regression_loss = coral_loss(regressionOutput.view(-1 , self.num_ranks - 1) , levels.view(-1 , self.num_ranks - 1))

    if self.mode.lower() == "both" and (labels != None and scores != None):
      totalLoss = (self.lambda_c * classification_loss) + (self.lambda_r * regression_loss)
    elif self.mode.lower() == "classification" and labels != None:
      totalLoss = classification_loss
    elif self.mode.lower() == "regression" and scores != None:
      totalLoss = regression_loss

    outputs = torch.concat((classificationOutput , regressionOutput) , dim=-1)

    finalClassificationOutput = torch.sigmoid(classificationOutput)
    finalRegressionOutput = torch.sigmoid(regressionOutput)

    finalClassificationOutput = proba_to_label(finalClassificationOutput.cpu().detach()).numpy()
    finalRegressionOutput = torch.sum(finalRegressionOutput.cpu().detach() , dim=-1).numpy() + 1

    return CustomOutput(
        loss=totalLoss,
        logits=outputs,
        classificationOutput=finalClassificationOutput,
        regressionOutput=finalRegressionOutput,
    )

In [16]:
def model_init(encoderPath=None,
               dimKey=None,
               customEncoder=None,
               customDim=None,
               mode="both",
               use_coral=True, 
               use_cls=True, 
               supportPooledRepresentation=False,
               freezeEmbedding=True, 
               num_labels=3, 
               num_ranks=5, 
               lambda_c=0.5, 
               lambda_r=0.5, 
               dropout_rate=0.2,):
  
  encoder = ts.AutoModel.from_pretrained(encoderPath) if encoderPath != None else customEncoder
  dim = encoder.config.to_dict()[dimKey] if dimKey != None else customDim

  model = SequenceClassificationModel(
                                      encoder,
                                      dim, 
                                      use_coral=use_coral, 
                                      use_cls=use_cls, 
                                      supportPooledRepresentation=supportPooledRepresentation, 
                                      mode=mode, 
                                      num_labels=num_labels, 
                                      num_ranks=num_ranks,
                                      lambda_c=lambda_c,
                                      lambda_r=lambda_r,
                                      dropout_rate=dropout_rate,
  )

  try:
    if freezeEmbedding:
      for param in model.encoder.embeddings.parameters():
        param.requires_grad = False
  except:
    print("The embedding layer name is different in this model, try to find the name of the emebdding layer and freeze it manually")
  
  return model

In [17]:
def makeTrainer(model, 
                trainDataset,
                data_collator,
                tokenizer, 
                outputsPath, 
                learning_rate=1.90323e-05, 
                scheduler="cosine",
                save_steps=5000,
                batch_size=8,
                num_epochs=5,
                weight_decay=0.00123974,
                roundingType="F"):

  def data_collator_fn(items , columns=[]):
    data_collator_input = {
        "input_ids": items[columns[0]],
        "attention_mask": items[columns[1]]
    }

    result = data_collator(data_collator_input)

    items[columns[0]] = result["input_ids"]
    items[columns[1]] = result["attention_mask"]

  def collate_function(items):
    outputDict = {
        key: [] for key in items[0].keys()
    }

    for item in items:
      for key in item.keys():
        outputDict[key].append(item[key])

    data_collator_fn(outputDict , ["input_ids" , "attention_mask"])

    #Removing unnecessary Items from outputDict
    columns = ["sentence" , "filler" , "id"]
    for item in columns:
      try:
        outputDict.pop(item)
      except:
        pass

    #Adding New Columns
    if "labels" in outputDict.keys():
      outputDict["labels"] = torch.tensor(outputDict.pop("labels"))
    if "scores" in outputDict.keys():
      if roundingType == "F":
        outputDict["scores"] = torch.tensor(outputDict.pop("scores") , dtype=torch.int32) - 1
      elif roundingType == "R":
        outputDict["scores"] = torch.tensor([round(score) for score in outputDict.pop("scores")] , dtype=torch.int32) - 1

    filler_indecies = torch.tensor(outputDict.pop("filler_indecies")).view(-1  , 1)
    outputDict["filler_indecies"] = (torch.arange(filler_indecies.shape[0]).view(-1 , 1) , filler_indecies)

    return outputDict 

  training_args = TrainingArguments(
      outputsPath, 
      learning_rate= learning_rate,
      lr_scheduler_type=scheduler,
      save_steps=save_steps,
      per_device_train_batch_size=batch_size,
      num_train_epochs=num_epochs,
      weight_decay=weight_decay,
      remove_unused_columns=False,
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=trainDataset,
      tokenizer=tokenizer,
      data_collator=collate_function,
  )

  return trainer , collate_function

In [18]:
model = model_init(encoderPath=PRE_TRAINED_MODEL,
                   dimKey=DIM_KEY,
                   mode="both", 
                   use_coral=True, 
                   use_cls=True, 
                   supportPooledRepresentation=False,
                   freezeEmbedding=True)

trainer , collate_function = makeTrainer(model,
                                         tokenizedTrainDataset,
                                         data_collator,
                                         tokenizer,
                                         "outputs/")

Downloading:   0%|          | 0.00/354M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.dense.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.bias', 'mask_predictions.classifier.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 19975
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 12485


Step,Training Loss
500,1.770100
1000,1.734400
1500,1.714600
2000,1.706400
2500,1.697700
3000,1.530300
3500,1.529400
4000,1.538100
4500,1.529600
5000,1.519900


Saving model checkpoint to content/outputs/checkpoint-5000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in content/outputs/checkpoint-5000/tokenizer_config.json
Special tokens file saved in content/outputs/checkpoint-5000/special_tokens_map.json
added tokens file saved in content/outputs/checkpoint-5000/added_tokens.json
Saving model checkpoint to content/outputs/checkpoint-10000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in content/outputs/checkpoint-10000/tokenizer_config.json
Special tokens file saved in content/outputs/checkpoint-10000/special_tokens_map.json
added tokens file saved in content/outputs/checkpoint-10000/added_tokens.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=12485, training_loss=1.4129978777267096, metrics={'train_runtime': 2149.8253, 'train_samples_per_second': 46.457, 'train_steps_per_second': 5.807, 'total_flos': 0.0, 'train_loss': 1.4129978777267096, 'epoch': 5.0})

#Evaluating on Val Dataset

In [26]:
def evaluateModel(
    model, 
    dataset, 
    collate_function,
):

  model.eval()

  #Passing the inputs through model
  labels = []
  scores = []

  for item in dataset:
    sample_input = collate_function([item])
    outputs = model(input_ids=sample_input["input_ids"].to(model.encoder.device),
                    attention_mask=sample_input["attention_mask"].to(model.encoder.device),
                    filler_indecies=sample_input["filler_indecies"],
                    scores=None)

    labels.append(outputs["classificationOutput"][0])
    scores.append(outputs["regressionOutput"][0])


  #Computing Accuracy
  count = 0
  correctCount = 0

  for prediction , target in zip(labels , dataset["labels"]):
    count += 1
    correctCount += 1 if prediction == target else 0

  accuracy = (correctCount / count)

  #Computing Spearman
  scores = np.array(scores , dtype=np.float32)
  valScores = np.array(dataset["scores"] , dtype=np.float32)
  spearman = stats.spearmanr(scores.reshape(-1 , 1) , valScores.reshape(-1 , 1))

  return (labels , scores) , accuracy , spearman

In [27]:
(labels , scores) , accuracy , spearman = evaluateModel(model, tokenizedValDataset, collate_function)

print(f"Accuracy is: {accuracy}")
print(f"Spearman is: {spearman}")

Accuracy is: 0.6328
Spearman is: SpearmanrResult(correlation=0.7310048129903207, pvalue=0.0)


#Making Predictions on Test Dataset

In [28]:
def predictOnTestDataset(
    model,
    dataset,
    collate_function,
    labelsPath=None,
    scoresPath=None,
):

  model.eval()

  ids = []
  classification_predictions = []
  ranking_predictions = []

  for item in dataset:
    sample_input = collate_function([item])

    outputs = model(input_ids=sample_input["input_ids"].to(model.encoder.device),
                    attention_mask=sample_input["attention_mask"].to(model.encoder.device),
                    filler_indecies=sample_input["filler_indecies"],
                    scores=None,
                    labels=None)

    ids.append(item["id"])
    classification_predictions.append(outputs["classificationOutput"][0])
    ranking_predictions.append(outputs["regressionOutput"][0])
  
  if labelsPath != None:
    open(labelsPath , mode="wb")
    write_predictions_to_file(labelsPath , ids , classification_predictions , "classification")
  
  if scoresPath != None:
    open(scoresPath , mode="wb")
    write_predictions_to_file(scoresPath , ids , ranking_predictions , "ranking")
  
  return ids , classification_predictions , ranking_predictions

In [29]:
ids , labels , scores = predictOnTestDataset(model , tokenizedTestDataset , collate_function , "classification_answers.tsv" , "ranking_answers.tsv" )

#Inference

In [30]:
def inference(
    model,
    sentences,
    fillers,
    tokenizer,
    collate_function
):

  model.eval()

  datasetDict = {
      "sentence": sentences,
      "filler": fillers,
  }

  dataset = Dataset.from_dict(datasetDict)
  tokenizedDataset = preprocessDataset(dataset , tokenizer)
  finalInput = collate_function(tokenizedDataset)

  outputs = model(
      input_ids=finalInput["input_ids"].to(model.encoder.device),
      attention_mask=finalInput["attention_mask"].to(model.encoder.device),
      filler_indecies=finalInput["filler_indecies"],
  )

  finalLabels = []

  for item in outputs["classificationOutput"].reshape(-1):
    if item == 0:
      finalLabels.append("Implausible")
    elif item == 1:
      finalLabels.append("Neutral")
    elif item == 2:
      finalLabels.append("Plausible")

  finalLabels = np.array(finalLabels)

  return {
      "labels": finalLabels,
      "scores": outputs["regressionOutput"],
  }
  

In [32]:
inference(model, 
          sentences=["This is a [MASK] to see how model works" , "This is a [MASK] to see how model works"], 
          fillers=["test" , "fork"],
          tokenizer=tokenizer,
          collate_function=collate_function,)

  0%|          | 0/2 [00:00<?, ?ex/s]

{'labels': array(['Plausible', 'Implausible'], dtype='<U11'),
 'scores': array([4.4753685, 1.7053134], dtype=float32)}